In [87]:
import glob
import os
import random as rn

import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline

In [88]:
file = open('../utils/envs', 'r')
lines = file.readlines()

repo_dir = seed = None
for line in lines:
       if 'LRDIR' in line:
           repo_dir = line.split('=')[-1]
       elif 'PYTHONHASHSEED' in line:
           seed = line.split('=')[-1]

os.environ['LRDIR'] = repo_dir
os.environ['PYTHONHASHSEED'] = seed

from utils import global_params

rn.seed(global_params.rn_seed)
np.random.seed(global_params.np_random_seed)
tf.random.set_seed(global_params.tf_random)

In [89]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [90]:
classes = global_params.classes
print('Classes:', classes)
classes_num = ['0' + str(i) if i < 10 else str(i) for i in range(1, len(classes) + 1)]
word_ids = ['0' + str(i) if i < 10 else str(i) for i in range(1, 11)]
classes_dict = dict(zip(classes_num, classes))
print('Classes dict:', classes_dict)

Classes: ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']
Classes dict: {'01': 'Begin', '02': 'Choose', '03': 'Connection', '04': 'Navigation', '05': 'Next', '06': 'Previous', '07': 'Start', '08': 'Stop', '09': 'Hello', '10': 'Web'}


In [91]:
repo_dir = os.environ['LRDIR'][:-1]
base_dir = os.path.join(os.environ['LRDIR'][:-1], 'lip_reading/data/')
test_dir = os.path.join(base_dir[:-1], 'test/')

def remove_dirs():
    try:
        for root, dirs, files in os.walk(test_dir, topdown=False):
            # for name in files:
            #     os.remove(os.path.join(root, name))
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        # os.rmdir(test_dir)
        print('Test directory cleared')
    except IOError as e:
        print('Error: ', e)


def create_dirs(prefix):
    for class_name in classes:
        test_vids_dir = os.path.join(test_dir, prefix + class_name)
        try:
            os.makedirs(test_vids_dir)
        except IOError as e:
            print('Error: ', e)
    print('Test directory filled')

remove_dirs()
create_dirs('')
create_dirs('_')

Test directory cleared
Test directory filled
Test directory filled


In [92]:
filenames = sorted(glob.glob("../data/test/*.mov"))
print(filenames[0])

../data/test/01_M01_01.mov


In [93]:
# 01_F05_01_color_001
for id in tqdm(range(len(filenames))):
    count = 1
    video_file = filenames[id]
    class_id = video_file.split('/')[-1].split('_')[0]
    video_name = video_file.split('/')[-1].split('.')[0]
    cap = cv2.VideoCapture(video_file)
    frameRate = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    while cap.isOpened():
        frame_id = cap.get(1) #current frame number
        success, frame = cap.read()
        if not success:
            break
        if not frame_id % 2:
            filename = '../data/test/' + '_' + classes_dict[class_id] + '/'\
                       + video_name + '_color_%s.jpg' % str(count).zfill(3)
            cv2.imwrite(filename, frame)
            count += 1

    cap.release()

100%|██████████| 10/10 [00:08<00:00,  1.17it/s]


In [94]:
predictor_path = global_params.predictor_path
detector = global_params.detector
predictor = global_params.predictor

size_x = 140
size_y = 70
scale = 100

In [95]:
# 001_01_F05_01_color_001
def extract_crop():
    for classi in tqdm(classes_num[:]):
        for f in sorted(glob.glob(os.path.join('../data/test/' + '_' + classes_dict[classi], '*.jpg'))):
            img = cv2.imread(f, 1)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            rects = detector(gray)

            for k, rect in enumerate(rects):
                shape = predictor(gray, rect)

                x_51 = shape.part(51).x
                y_51 = shape.part(51).y
                x_57 = shape.part(57).x
                y_57 = shape.part(57).y

                x1_m = x_51 - 18
                y1_m = y_51 - 7
                x2_m = x_57 + 18
                y2_m = y_57 + 9

                offset_x_m = (size_x - (abs(x1_m - x2_m))) / 2
                offset_y_m = (size_y - (abs(y1_m - y2_m))) / 2

                img = img[int(y1_m - offset_y_m):int(y2_m + offset_y_m),
                      int(x1_m - offset_x_m):int(x2_m + offset_x_m)]

            savename = '../data/test/' + classes_dict[classi] + '/' + f.split('/')[-1]
            cv2.imwrite(savename, img)

extract_crop()

100%|██████████| 10/10 [00:33<00:00,  3.30s/it]
